In [1]:

from auditory_cortex.Regression import transformer_regression

reg = transformer_regression('/depot/jgmakin/data/auditory_cortex/josh_data/data',"200213")

Data from 64 channels loaded...!
Objects created, now loading Transformer layer features...!


/home/akamsali/anaconda3/envs/research_env/lib/python3.8/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /opt/conda/conda-bld/pytorch_1631630815121/work/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [9]:

# channels = np.arange(0,reg.dataset.num_channels).tolist()
num_layers = len(reg.layers)


PCt =[]
PCv =[]
PCtt =[]
w = 100
ch=1
sub = "200213"

corr_values = {}
for ch in range(4):
    R2t =[]  
    R2v =[] 
    R2tt =[]
    for l in range(num_layers):
        r2t, r2v,r2tt = reg.get_cc_norm(l,w,channel=ch, delay=0)
        R2t.append(r2t.item())
        R2v.append(r2v.item())
        R2tt.append(r2tt.item())
        # PCt.append(pct)
        # PCv.append(pcv)
        # PCtt.append(pctt)
    corr_values[ch] =  {"train": R2t, "val": R2v, "test": R2tt}
# fig, ax = plt.subplots(1,2, figsize=(14,6), sharey=True)

with open("/scratch/gilbreth/akamsali/Research/Makin/Auditory_Cortex/"+sub + "_" + str(w), 'a') as f:
    json.dump(corr_values, f)

    
# plt.plot(R2t, label='Training')
# plt.plot(R2v, label='Val')
# plt.plot(R2tt, label='Test')
# plt.legend()


# ax[0].plot(R2t, label='Training')
# ax[0].plot(R2v, label='Val')
# ax[0].plot(R2tt, label='Test')
# ax[0].legend()
# ax[0].set_title("R2")


# ax[1].plot(PCt, label='Training')
# ax[1].plot(PCv, label='Val')
# ax[1].plot(PCtt, label='Test')
# ax[1].legend()
# ax[1].set_title("PC")

# plt.title("200213, bin_width="+ str(w))
# # plt.savefig("/Users/akshita/Documents/Research/Makin/outputs/200213_" + str(w))
# plt.show()

In [ ]:
import numpy as np

from Dataset import Neural_Data


neural_data = Neural_Data('/Users/akshita/Documents/Research/Makin/data',"200206")

In [ ]:
repeated_sents = {}
for i in range(499):
    data = neural_data.retrieve_spikes_count_for_all_trials(i, w=100)
    if data[1].shape[0] > 1:
        repeated_sents[i] = data[1].shape
print(repeated_sents)

In [ ]:
from scipy import stats
import matplotlib.pyplot as plt
import csv
# print(channel.shape)

# sents = [12,13,32,43,56,163,212,218,287,308]
sents = [12]
# sent_vals = {}
# wins = [1,5,10,20,30,40,50,75,100,200]
wins = [10]
for w in wins:
    for sent in sents:
        data = neural_data.retrieve_spikes_count_for_all_trials(sent, w=w)

        for j in range(len(data)):
            channel = data[j]
            # print(channel.shape)
            cnt = 0
            val = []
            p_r = []
            for shift in [0]:#1, 5, 10]:
                for i in range(11):
                    cnt+=1.0
                    ind = np.arange(0,11,1)
                    b_ind = ind[i]
                    a_ind = np.delete(ind, i)
                    # print(a_ind, b_ind)
                    a = np.mean(channel[a_ind, :], axis=0)
                    b = channel[b_ind,:]
                    b = np.roll(b, shift)
                    
                    res = stats.linregress(a, b)
                    pearson_r = stats.pearsonr(a, b)
                    val.append(res.rvalue**2)
                    p_r.append(pearson_r[0]**2)

                with open("/home/amy/Documents/Research/Makin/outputs/good_sessions/linreg_cs_win_" + str(w)+ "_"+ str(sent) +"_" + str(shift)+".csv", 'a') as f:
                    writer=csv.writer(f)
                    row = val 
                    writer.writerow(row)
                    f.close()
    print("done: ", w)


In [ ]:
ind = np.arange(0,11,1)
b_ind = ind[i]
a_ind = np.delete(ind, i)
print(a_ind, b_ind)

In [ ]:
import pandas as pd
fig, ax = plt.subplots(4, len(wins),sharey=True,figsize=(5,6))

for i, shift in enumerate([0, 1, 5, 10]):
    for j, w in enumerate(wins):
        for sent in sents:
        # plt.figure()
            d = pd.read_csv("/home/amy/Documents/Research/Makin/outputs/good_sessions/linreg_cs_win_" + str(w)+ "_"+ str(sent) +"_" + str(shift)+ ".csv", header=None)
            d.columns = ["channels", "reg_r2", "pearson_r2"]

            avg_r2 = d['reg_r2'].to_numpy()
            pearson_r2 = d['pearson_r2'].to_numpy()
            # print(avg_r2.shape, pearson_r2.shape)

            # channels = d['channels'].to_numpy()
            # plt.figure()
            # ax[i, j].plot(channels, avg_r2, label=str(sent))
            ax[i].plot(avg_r2-pearson_r2, label=str(sent))
            ax[0].set_title(f"Win={w}")
            ax[i].set_ylabel(f"Shift={shift}")
            plt.ylim(-1.25, 0.8)
        # plt.xlabel("channels")
    # plt.title(f"Win = {w}" , fontsize=16)plt.legend()
        # plt.savefig("/home/amy/Documents/Research/Makin/outputs/correlation/linreg_cs_win_" + str(w)+ ".png")
plt.show()

In [ ]:
for j, w in enumerate(wins):
    for i, shift in enumerate([0,1, 5, 10]):
        plt.figure()
        for sent in sents:
            d = pd.read_csv("/home/amy/Documents/Research/Makin/outputs/correlation/linreg_cs_win_" + str(w)+ "_"+ str(sent) +"_" + str(shift)+ ".csv", header=None)
            d.columns = ["channels", "reg_r2", "pearson_r2"]

            avg_r2 = d['reg_r2'].to_list()
            pearson_r2 = d['pearson_r2'].to_list()
            channels = d['channels'].to_list()
            # plt.figure()
            plt.plot(channels, avg_r2, label=str(sent))
            plt.title(f"Win={w}, Shift={shift}")
            # ax[i,j].set_ylabel(f"Shift={shift}")
            plt.ylim(-0.5, 0.4)
        # plt.xlabel("channels")
    # plt.title(f"Win = {w}" , fontsize=16)plt.legend()
        # plt.savefig("/home/amy/Documents/Research/Makin/outputs/correlation/linreg_cs_win_" + str(w)+ ".png")
        plt.show()

In [ ]:
import pandas as pd

for sent in sents:
    d2 = pd.read_csv("/home/amy/Documents/Research/Makin/outputs/correlation/linreg_loo_win_ms10_"+ str(sent) +".csv", header=None)
    d2.columns = ["channels", "avg_r2"]

    avg_R2 = d2['avg_r2'].to_list()
    channels = d2['channels'].to_list()
    
    plt.plot(channels, avg_R2 ,label=str(sent))
    plt.ylim(-1.25, 0.8)
plt.title("Correlation for sentences ms vs channels, w=10", fontsize=16)
plt.legend()
plt.show()

In [ ]:
import numpy as np

a = np.zeros(100)
b = np.zeros(100)
ind_a = np.random.randint(0,100,10)
ind_b = np.random.randint(0,100,10)

a[ind_a] = 1    
b[ind_b] = 1
res = stats.linregress(a,b)

res.rvalue**2

In [ ]:
for w in [1]:
    for sent in [12]:
        data = neural_data.retrieve_spikes_count_for_all_trials(sent, w=w)

        for j in [20]:
            channel = data[j]
            # for shift in [1, 5, 10]:
            for i in range(1):
                plt.plot(channel[i], label=j)

    plt.legend()
    plt.show()

In [1]:
from Dataset import Neural_Data

dataset = Neural_Data("/depot/jgmakin/data/auditory_cortex", "200206")

Data from 62 channels loaded...!


In [2]:
data = dataset.audio(0)
data.shape

(30618,)

In [ ]:
import numpy as np
import torch
import os
from scipy import linalg
from transformers import Speech2TextForConditionalGeneration, Speech2TextProcessor
from Dataset import Neural_Data
from Feature_Extractors import Feature_Extractor_S2T
from sklearn.decomposition import PCA

from deepspeech_pytorch.model import DeepSpeech

import matplotlib.pyplot as plt

class transformer_regression():
  def __init__(self, dir, subject):
    self.dir = os.path.join(dir, subject)
    self.dataset = Neural_Data(dir, subject)
    # self.layers = ["model.encoder.layers.0.fc2", "model.encoder.layers.1.fc2", "model.encoder.layers.2.fc2","model.encoder.layers.3.fc2",
    #                "model.encoder.layers.4.fc2","model.encoder.layers.5.fc2","model.encoder.layers.6.fc2","model.encoder.layers.7.fc2",
    #                "model.encoder.layers.8.fc2","model.encoder.layers.9.fc2"]
    # self.model = Speech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr")
    # self.processor = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr")
    self.layers = ["conv.seq_module.0"]
    self.model = DeepSpeech.load_from_checkpoint('/scratch/gilbreth/akamsali/Research/Makin/data/pretrained/librispeech_pretrained_v3.ckpt')
    self.model_extractor = Feature_Extractor_S2T(self.model, self.layers)
    print("Objects created, now loading Transformer layer features...!")
    self.features, self.demean_features = self.get_transformer_features()

  def simply_spikes(self, sent_s=1, sent_e=499, ch=0, w = 40, delay=0):
    spikes ={}
    for x,i in enumerate(range(sent_s,sent_e)):
      spikes[x] = torch.tensor(self.dataset.retrieve_spike_counts(sent=i, win=w, delay=delay ,early_spikes=False)[ch])
    spikes = torch.cat([spikes[i] for i in range(sent_e - sent_s)], dim = 0).numpy()
    return spikes

  def demean_spikes(self, sent_s=1, sent_e=499, ch=0, w = 40):
    spikes ={}
    spk_mean = {}
    for x,i in enumerate(range(sent_s,sent_e)):
      spikes[x] = torch.tensor(self.dataset.retrieve_spike_counts(sent=i, win=40 ,early_spikes=False)[ch])
      spk_mean[x] = torch.mean(spikes[x], dim = 0)
      spikes[x] = spikes[x] - spk_mean[x]
    spikes = torch.cat([spikes[i] for i in range(sent_e - sent_s)], dim = 0).numpy()
    return spikes



  def benchmark_r2_score(self, w = 40, sent = 12):
    #These sentences have repeated trials...!
    #sents = [12,13,32,43,56,163,212,218,287,308]
    r2_scores = np.zeros(self.dataset.num_channels)
    #trials = obj.dataset.get_trials(13)
    spkk = self.dataset.retrieve_spike_counts_for_all_trials(sent=sent, w=w)
  
    for i in range(self.dataset.num_channels):
      h1 = np.mean(spkk[i][0:6], axis=0)
      h2 = np.mean(spkk[i][6:], axis=0)
      r2_scores[i] = self.r2(h1,h2)
    return r2_scores

  def translate(self, aud, fs = 16000):
    inputs_features = self.processor(aud,padding=True, sampling_rate=fs, return_tensors="pt").input_features
    generated_ids = self.model_extractor(inputs_features)

  # def simply_stack(self, features):
  #   features = torch.cat([features[i] for i in range(498)], dim=0)
  #   return features

  # def get_transformer_features(self):
  #   sent_s = 1
  #   sent_e = 499
  #   features = [{} for _ in range(len(self.layers))]

  #   feats = {}
  #   for x, i in enumerate(range(sent_s, sent_e)):
  #     self.translate(self.dataset.audio(i))
  #     for j, l in enumerate(self.layers):
  #       features[j][x] = self.model_extractor.features[l]
    
  #   for j, l in enumerate(self.layers):
  #     feats[j] = torch.cat([features[j][i] for i in range(sent_e-sent_s)], dim=0).numpy()
  #   return feats

  def get_transformer_features(self, sent_s = 1, sent_e = 499):
    
    features = [{} for _ in range(len(self.layers))]
    demean_features = [{} for _ in range(len(self.layers))]
    f_mean = {}    
    feats = {}
    demean_feats = {}
    for x, i in enumerate(range(sent_s, sent_e)):
      self.translate(self.dataset.audio(i))
      for j, l in enumerate(self.layers):
        features[j][x] = self.model_extractor.features[l]
        f_mean[x] = torch.mean(features[j][x], dim = 0)    
        demean_features[j][x]= features[j][x] - f_mean[x]
    for j, l in enumerate(self.layers):
      feats[j] = torch.cat([features[j][i] for i in range(sent_e-sent_s)], dim=0).numpy()
      demean_feats[j] = torch.cat([demean_features[j][i] for i in range(sent_e-sent_s)], dim=0).numpy()
    return feats, demean_feats

  # for i in range(498):
  #   f_mean[i] = torch.mean(features[i], dim = 0)
  #   features[i] = features[i] - f_mean[i]
  # features = torch.cat([features[i] for i in range(498)], dim=0)

  def down_sample_features(self, feats, k):
    out = np.zeros((int(np.ceil(feats.shape[0]/k)),feats.shape[1]))
    for i in range(out.shape[0]):
      #Just add the remaining samples at the end...!
      if (i == out.shape[0] -1):
        out[i] = feats[k*i:, :].sum(axis=0)
      else:  
        out[i] = feats[k*i:k*(i+1), :].sum(axis=0)
    return out

  def down_sample_spikes(self, spks, k):
    out = np.zeros(int(np.ceil(spks.shape[0]/k)))
    for i in range(out.shape[0]):
      #Just add the remaining samples at the end...!
      if (i == out.shape[0] -1):
        out[i] = spks[k*i:].sum(axis=0)
      else:  
        out[i] = spks[k*i:k*(i+1)].sum(axis=0)
    return out

  def compute_r2(self, layer, win):
    k = int(win/40)    # 40 is the min, bin size for 'Speech2Text' transformer model 
    # print(f"k = {k}")
    r2t = np.zeros(self.dataset.num_channels)
    r2v = np.zeros(self.dataset.num_channels)
    pct = np.zeros(self.dataset.num_channels)
    pcv = np.zeros(self.dataset.num_channels)

    #downsamples if k>1 
    if k >1:
      feats = self.down_sample_features(self.features[layer], k)
    else:
      feats = self.features[layer]

    m = int(feats.shape[0] *0.75)
    x_train = feats[0:m, :]
    x_test = feats[m:, :]
    
    for i in range(self.dataset.num_channels):
      y = self.simply_spikes(ch=i)
      if k>1:
        y = self.down_sample_spikes(y,k)
      y_train = y[0:m]
      y_test = y[m:]
      B = self.regression_param(x_train, y_train)
 
      r2t[i] = self.regression_score(x_train, y_train, B)
      r2v[i] = self.regression_score(x_test, y_test, B)
      pct[i] = (np.corrcoef(self.predict(x_train, B), y_train)[0,1])**2
      pcv[i] = (np.corrcoef(self.predict(x_test, B), y_test)[0,1])**2
    return r2t, r2v, pct, pcv



  def compute_r2_channel(self, layer, win, channel, delay):
    k = int(win/40)    # 40 is the min, bin size for 'Speech2Text' transformer model 
    # print(f"k = {k}")
    r2t = np.zeros(1)
    r2v = np.zeros(1)
    r2tt = np.zeros(1)
    pct = np.zeros(1)
    pcv = np.zeros(1)
    pctt = np.zeros(1)

    #downsamples if k>1 
    if k >1:
      feats = self.down_sample_features(self.features[layer], k)
    else:
      feats = self.features[layer]

    n1 = int(feats.shape[0] *0.75)
    n2 = int(feats.shape[0] *0.90)
    x_train = feats[0:n1, :]
    x_val = feats[n1:n2, :]
    x_test = feats[n2:, :]
    
    # for i in range(self.dataset.num_channels):
    y = self.simply_spikes(ch=channel, delay=delay)
    if k>1:
      y = self.down_sample_spikes(y,k)
    y_train = y[0:n1]
    y_val = y[n1:n2]    
    y_test = y[n2:]
    B = self.regression_param(x_train, y_train)

    r2t = self.regression_score(x_train, y_train, B)
    r2v = self.regression_score(x_val, y_val, B)
    r2tt = self.regression_score(x_test, y_test, B)
    pct = np.corrcoef(self.predict(x_train, B), y_train)
    pcv = np.corrcoef(self.predict(x_val, B), y_val)
    pctt = np.corrcoef(self.predict(x_test, B), y_test)
    pct = np.square(pct[0,1])
    pcv = np.square(pcv[0,1])
    pctt = np.square(pctt[0,1])
    
    return r2t, r2v,r2tt, pct, pcv,pctt
  
  def FE_r2_channel(self, layer, win, channel):
    k = int(win/40)    # 40 is the min, bin size for 'Speech2Text' transformer model 
    print(f"k = {k}")
    r2t = np.zeros(1)
    r2v = np.zeros(1)
    pct = np.zeros(1)
    pcv = np.zeros(1)
    #downsamples if k>1 
    if k >1:
      feats = self.down_sample_features(self.demean_features[layer], k)
    else:
      feats = self.demean_features[layer]

    m = int(feats.shape[0] *0.75)
    x_train = feats[0:m, :]
    x_test = feats[m:, :]
    
    # for i in range(self.dataset.num_channels):
    y = self.demean_spikes(ch=channel)
    if k>1:
      y = self.down_sample_spikes(y,k)
    y_train = y[0:m]
    y_test = y[m:]
    B = self.regression_param(x_train, y_train)

    r2t = self.regression_score(x_train, y_train, B)
    r2v = self.regression_score(x_test, y_test, B)
    pct = (np.corrcoef(self.predict(x_train, B), y_train)[0,1])**2
    pcv = (np.corrcoef(self.predict(x_test, B), y_test)[0,1])**2
    return r2t, r2v, pct, pcv


  def r2(self, labels, predictions):
    score = 0.0
    mean = np.mean(labels)
    denom = np.sum(np.square(labels - mean))
    num = np.sum(np.square(labels - predictions))
    score = 1 - num/denom
    return score
  def regression_score(self, X,y, B):
    y_hat = self.predict(X,B)
    return self.r2(y, y_hat)

  def regression_param(self, X, y):
    B = linalg.lstsq(X, y)[0]
    return B
  def predict(self, X, B):
    return X@B

  def get_pcs(self, layer, sents):
      #layer = 0
      #sents = [495, 496, 497]
      feats = [{} for _ in sents]
      feats_pcs = {}
      for i, s in enumerate(sents):
        feats[i], _ = self.get_transformer_features(s, s+1)
      layer_features = self.features[layer]
      m = layer_features.shape[0]
      pc = PCA(n_components=2)
      pc.fit(layer_features[:int(0.75*m),:])
      for i, s in enumerate(sents):
        feats_pcs[i] = pc.transform(feats[i][layer]) 
      return feats_pcs

  def plot_pcs(self, l=0, sents=[495,496,497]):
      c_maps = ['Greys', 'Purples', 'Blues', 'Oranges',
                'Greens', 'Reds',
                #'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
                #'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn'
                ] 
      leg_colors = ['tab:gray','tab:purple','tab:blue','tab:orange',
                    'tab:green','tab:red']

      pcs = self.get_pcs(layer=l, sents=sents)

      for i in range(len(pcs.keys())):
        shades = np.arange(0,pcs[i].shape[0])

        plt.scatter(pcs[i][:,0], pcs[i][:,1], label=f"sent: {sents[i]}", cmap=c_maps[(i+2)%len(c_maps)], c=shades, vmin=-40, vmax=80)

      leg = plt.legend(loc='best')
      for i in range(len(pcs.keys())):
        leg.legendHandles[i].set_color(leg_colors[(i+2)%len(leg_colors)])

      plt.xlabel(f"PC1")
      plt.ylabel(f"PC2")
      plt.title(f"{self.layers[l]}")